In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
from core.dprocessing import *
from core.optimizers import *
from core.utils import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import classification_report

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [4]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)
labels = dataset.iloc[:, [984]]

In [5]:
features = ['integrals_pos', 'min_trough2peak', 'integrals', 'envelope', 'oscpower', 'fold_change', 'valley_amps']
time_steps = [98, None, 98, 25, None, 98, 15]

In [6]:
df = pd.read_csv('C:/Users/minha/Research/hoffmanlab/featuresel/data/untuned_model_feature_importance.csv')
feature_importance = df.values.tolist()

#### Training a model on all time series per feature for top 7 features
* integrals_pos
* min_trough2peak
* integrals
* envelope
* oscpower
* fold_change
* valley_amps

In [ ]:
dataset['integrals_pos_1']

In [ ]:
feature_name = 'integrals_pos'

In [ ]:
df = pd.DataFrame()
for i in range(1, 99):
    column = feature_name + '_' + str(i)
    column = dataset[column]
    df = pd.concat([df, column], axis=1)

In [ ]:
# pass in list of feature names, pass in number of time steps for that corresponding feature

In [ ]:
test2 = process(features, time_steps, dataset)

In [ ]:
test2

In [ ]:
model = xgb.XGBClassifier()

In [ ]:
labels = dataset.iloc[:, [984]]

In [ ]:
data, labels = test2.to_numpy(), labels.to_numpy()

In [ ]:
labels = labels.reshape((-1, ))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size = 0.1, random_state = 42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_val)

In [ ]:
pred

In [ ]:
y_val

In [ ]:
x = 0
for y_pred, y_true in zip(pred, y_val):
    if y_pred == y_true:
        x += 1
print(x / len(y_val))

In [ ]:
# almost same accuracy vs trained on all features (985), removed a lot of noise ig

In [ ]:
cr = classification_report(y_val, pred, target_names=ligands)

In [ ]:
print(cr)

### given the entire array of the top features:
* parameters:
    * distinct features
    * upper bound (search in the upper x% of the array) for time steps for THOSE features

### trying 25% of top features for now:
* loop through all partitions and find the best?

In [ ]:
data = partition_features(features, time_steps, feature_importance, 0.25, dataset)

In [ ]:
data

In [ ]:
# data = data.to_numpy()
# labels = labels.to_numpy()

In [ ]:
labels = labels.reshape((-1, ))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_val)

In [ ]:
cr = classification_report(y_val, pred, target_names=ligands)

In [ ]:
print(cr)

### features trained on top 25% of timesteps for top 7 features yields 63% accuracy, 1% lower than model trained on 985 features

### make function to loop thru multiple bounds (partitions), find most optimal split (# of time steps for each feature)
* maybe try different partitions for each feature instead of partitioning then grabbing the time steps?
    * ie: taking top 10% time steps for integrals, integrals_pos, etc rather than splitting dataset into top 10% then finding time steps
* could also try developing some method of using quantitative values to decide which time steps to use (xgboost returns numeric values as weights for each feature)

In [7]:
interval = 10
initial_bound = 0.5
initial_bound_2 = 1

In [8]:
optimize_features(features, time_steps, feature_importance, dataset, initial_bound, interval)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[01:25:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|████████▎                                                                          | 1/10 [00:10<01:31, 10.19s/it]

              precision    recall  f1-score   support

         CpG       0.43      0.50      0.46       198
         FLA       0.44      0.35      0.39       156
         FSL       0.61      0.55      0.58       207
         LPS       0.56      0.63      0.59       175
         P3K       0.60      0.49      0.54       175
         PIC       0.63      0.70      0.66       120
         R84       0.81      0.86      0.84       264
         TNF       0.63      0.62      0.62       143

    accuracy                           0.60      1438
   macro avg       0.59      0.59      0.59      1438
weighted avg       0.60      0.60      0.60      1438

[01:25:35] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|████████████████▌                                                                  | 2/10 [00:20<01:21, 10.21s/it]

...

              precision    recall  f1-score   support

         CpG       0.50      0.50      0.50       196
         FLA       0.47      0.40      0.43       167
         FSL       0.53      0.53      0.53       180
         LPS       0.58      0.65      0.62       186
         P3K       0.52      0.46      0.49       137
         PIC       0.70      0.74      0.72       138
         R84       0.84      0.85      0.84       258
         TNF       0.70      0.74      0.72       176

    accuracy                           0.62      1438
   macro avg       0.61      0.61      0.61      1438
weighted avg       0.62      0.62      0.62      1438

[01:26:47] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:32<00:10, 10.32s/it]

              precision    recall  f1-score   support

         CpG       0.52      0.50      0.51       213
         FLA       0.35      0.32      0.33       146
         FSL       0.55      0.56      0.55       186
         LPS       0.54      0.60      0.57       172
         P3K       0.50      0.44      0.47       144
         PIC       0.66      0.65      0.66       150
         R84       0.80      0.86      0.83       259
         TNF       0.70      0.70      0.70       168

    accuracy                           0.60      1438
   macro avg       0.58      0.58      0.58      1438
weighted avg       0.59      0.60      0.60      1438

[01:26:57] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:42<00:00, 10.30s/it]

              precision    recall  f1-score   support

         CpG       0.52      0.58      0.55       210
         FLA       0.46      0.37      0.41       164
         FSL       0.54      0.51      0.53       181
         LPS       0.56      0.62      0.59       172
         P3K       0.59      0.46      0.52       156
         PIC       0.68      0.71      0.70       131
         R84       0.79      0.89      0.84       263
         TNF       0.71      0.68      0.69       161

    accuracy                           0.62      1438
   macro avg       0.61      0.60      0.60      1438
weighted avg       0.61      0.62      0.61      1438



In [9]:
optimize_features(features, time_steps, feature_importance, dataset, initial_bound_2, interval)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[01:27:08] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|████████▎                                                                          | 1/10 [00:16<02:24, 16.08s/it]

              precision    recall  f1-score   support

         CpG       0.50      0.57      0.53       191
         FLA       0.54      0.43      0.48       152
         FSL       0.55      0.47      0.50       173
         LPS       0.57      0.66      0.61       183
         P3K       0.55      0.54      0.54       139
         PIC       0.79      0.78      0.78       169
         R84       0.81      0.85      0.83       278
         TNF       0.68      0.65      0.67       153

    accuracy                           0.64      1438
   macro avg       0.62      0.62      0.62      1438
weighted avg       0.64      0.64      0.64      1438

[01:27:24] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|████████████████▌                                                                  | 2/10 [00:32<02:08, 16.09s/it]

...

              precision    recall  f1-score   support

         CpG       0.48      0.58      0.53       191
         FLA       0.53      0.45      0.49       147
         FSL       0.62      0.59      0.61       199
         LPS       0.55      0.65      0.60       178
         P3K       0.56      0.40      0.47       163
         PIC       0.74      0.77      0.76       141
         R84       0.85      0.84      0.85       251
         TNF       0.67      0.70      0.69       168

    accuracy                           0.63      1438
   macro avg       0.63      0.62      0.62      1438
weighted avg       0.64      0.63      0.63      1438

[01:29:16] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:23<00:15, 15.90s/it]

              precision    recall  f1-score   support

         CpG       0.51      0.57      0.54       204
         FLA       0.51      0.42      0.46       165
         FSL       0.59      0.50      0.54       197
         LPS       0.63      0.72      0.67       174
         P3K       0.54      0.53      0.54       149
         PIC       0.79      0.76      0.78       136
         R84       0.85      0.84      0.84       259
         TNF       0.63      0.70      0.66       154

    accuracy                           0.64      1438
   macro avg       0.63      0.63      0.63      1438
weighted avg       0.64      0.64      0.64      1438

[01:29:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:39<00:00, 15.93s/it]

              precision    recall  f1-score   support

         CpG       0.52      0.55      0.54       203
         FLA       0.57      0.44      0.50       173
         FSL       0.54      0.60      0.57       186
         LPS       0.60      0.73      0.66       175
         P3K       0.61      0.48      0.53       155
         PIC       0.64      0.71      0.67       134
         R84       0.85      0.84      0.84       257
         TNF       0.66      0.64      0.65       155

    accuracy                           0.63      1438
   macro avg       0.62      0.62      0.62      1438
weighted avg       0.63      0.63      0.63      1438



### valley_amps actually should be replaced by time_series

In [20]:
features = ['integrals_pos', 'min_trough2peak', 'integrals', 'time_series', 'envelope', 'oscpower', 'fold_change']
time_steps = [98, None, 98, 98, 25, None, 98, 15]

In [21]:
data = partition_features(features, time_steps, feature_importance, 0.25, dataset)

In [22]:
data = data.to_numpy()
labels = labels.to_numpy()

In [23]:
labels = labels.reshape((-1, 1))

In [24]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)

In [25]:
new_model = xgb.XGBClassifier(use_label_encoder=False)

In [26]:
new_model.fit(X_train, y_train)

[16:13:56] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [27]:
pred = new_model.predict(X_val)

In [36]:
cr = classification_report(y_val, pred, target_names=ligands, output_dict=True)

In [43]:
x = cr['macro avg']['f1-score']

In [41]:
d = {}

In [44]:
d[x] = 0.1

In [46]:
d[1] = 0.2

In [56]:
print(d.get(max(d)))

0.2


### replacing valley_amps with time_series doesn't make much of a difference
* certain features seem to improve classification of certain ligands
    * replacing valley_amps with time_series improves TNF but worsens almost everything else

### try fitting models on only time_series data?

### notes:
* features1/timestep1 : vector of 7 time-series only features
* features2/timestep2 : vector of 6 time-series only features
* features3/timestep3 : features2/timestep2 + min_trough2peak
* features4/timestep4 : top 7 features in feature importance
* features5/timestep5 : features1/timestep1 + min_trough2peak
* features6/timestep6 : features1/timestep1 + min_trough2peak + oscpower

In [7]:
features1 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps', 'derivatives']
timestep1 = [98, 98, 98, 25, 98, 15, 96] 

In [8]:
features2 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps']
timestep2 = [98, 98, 98, 25, 98, 15] 

In [9]:
features3 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps', 'min_trough2peak']
timestep3 = [98, 98, 98, 25, 98, 15, None] 

In [10]:
features4 = ['integrals_pos', 'min_trough2peak', 'integrals', 'time_series', 'envelope', 'oscpower', 'fold_change']
timestep4 = [98, None, 98, 98, 25, None, 98, 15]

In [11]:
features5 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps', 'derivatives', 'min_trough2peak']
timestep5 = [98, 98, 98, 25, 98, 15, 96, None] 

In [12]:
features6 = ['integrals_pos', 'integrals', 'time_series', 'envelope', 'fold_change', 'valley_amps', 'derivatives', 'min_trough2peak', 'oscpower']
timestep6 = [98, 98, 98, 25, 98, 15, 96, None, None] 

In [10]:
single_fit(features1, timestep1, feature_importance, 0.25, dataset, labels)

[15:48:35] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.50      0.57      0.53       196
         FLA       0.52      0.48      0.50       143
         FSL       0.60      0.62      0.61       189
         LPS       0.60      0.74      0.66       149
         P3K       0.64      0.51      0.57       185
         PIC       0.75      0.70      0.73       165
         R84       0.80      0.82      0.81       249
         TNF       0.73      0.67      0.70       162

    accuracy                           0.65      1438
   macro avg       0.64      0.64      0.64      1438
weighted avg       0.65      0.65      0.65      1438



In [11]:
single_fit(features2, timestep2, feature_importance, 0.25, dataset, labels)

[15:49:44] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.48      0.56      0.52       196
         FLA       0.51      0.44      0.47       143
         FSL       0.57      0.57      0.57       189
         LPS       0.58      0.77      0.66       149
         P3K       0.64      0.51      0.57       185
         PIC       0.77      0.67      0.72       165
         R84       0.78      0.82      0.80       249
         TNF       0.73      0.67      0.70       162

    accuracy                           0.63      1438
   macro avg       0.63      0.63      0.63      1438
weighted avg       0.64      0.63      0.63      1438



In [12]:
single_fit(features3, timestep3, feature_importance, 0.25, dataset, labels)

[15:50:23] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.52      0.55      0.54       196
         FLA       0.52      0.45      0.48       143
         FSL       0.61      0.59      0.60       189
         LPS       0.58      0.77      0.66       149
         P3K       0.62      0.51      0.56       185
         PIC       0.80      0.72      0.76       165
         R84       0.76      0.85      0.80       249
         TNF       0.74      0.68      0.71       162

    accuracy                           0.65      1438
   macro avg       0.64      0.64      0.64      1438
weighted avg       0.65      0.65      0.65      1438



In [14]:
single_fit(features4, timestep4, feature_importance, 0.25, dataset, labels)

[15:52:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.45      0.54      0.49       196
         FLA       0.49      0.46      0.48       143
         FSL       0.56      0.57      0.56       189
         LPS       0.57      0.73      0.64       149
         P3K       0.61      0.46      0.52       185
         PIC       0.78      0.70      0.74       165
         R84       0.77      0.80      0.79       249
         TNF       0.73      0.64      0.68       162

    accuracy                           0.62      1438
   macro avg       0.62      0.61      0.61      1438
weighted avg       0.63      0.62      0.62      1438



In [17]:
single_fit(features5, timestep5, feature_importance, 0.25, dataset, labels)

[15:55:24] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.50      0.58      0.53       196
         FLA       0.55      0.48      0.51       143
         FSL       0.60      0.62      0.61       189
         LPS       0.60      0.71      0.65       149
         P3K       0.65      0.55      0.59       185
         PIC       0.78      0.72      0.75       165
         R84       0.79      0.83      0.81       249
         TNF       0.73      0.65      0.69       162

    accuracy                           0.65      1438
   macro avg       0.65      0.64      0.64      1438
weighted avg       0.66      0.65      0.65      1438



In [18]:
single_fit(features6, timestep6, feature_importance, 0.25, dataset, labels)

[15:56:09] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         CpG       0.53      0.59      0.56       196
         FLA       0.55      0.49      0.52       143
         FSL       0.57      0.61      0.59       189
         LPS       0.59      0.72      0.65       149
         P3K       0.64      0.52      0.57       185
         PIC       0.78      0.72      0.75       165
         R84       0.77      0.83      0.80       249
         TNF       0.76      0.64      0.70       162

    accuracy                           0.65      1438
   macro avg       0.65      0.64      0.64      1438
weighted avg       0.65      0.65      0.65      1438



### add in oscpower and mintrough2_peak to see if performance increases more?

### so far:
* changing bounds hardly affects performance, focus on actual feature selection instead?

In [13]:
d = optimize_timesteps(features1, timestep1, feature_importance, dataset, 10, labels, printall=True, printmax=False)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[16:35:55] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|████████▏                                                                         | 1/10 [02:03<18:27, 123.02s/it]

              precision    recall  f1-score   support

         CpG       0.51      0.59      0.55       196
         FLA       0.54      0.46      0.50       143
         FSL       0.56      0.62      0.59       189
         LPS       0.63      0.73      0.67       149
         P3K       0.61      0.49      0.54       185
         PIC       0.73      0.68      0.71       165
         R84       0.80      0.82      0.81       249
         TNF       0.73      0.67      0.70       162

    accuracy                           0.64      1438
   macro avg       0.64      0.63      0.63      1438
weighted avg       0.65      0.64      0.64      1438

[16:37:58] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|████████████████▍                                                                 | 2/10 [04:02<16:07, 120.94s/it]

...

              precision    recall  f1-score   support

         CpG       0.51      0.61      0.55       196
         FLA       0.51      0.46      0.48       143
         FSL       0.60      0.63      0.62       189
         LPS       0.59      0.70      0.64       149
         P3K       0.66      0.54      0.60       185
         PIC       0.78      0.72      0.75       165
         R84       0.81      0.82      0.82       249
         TNF       0.73      0.66      0.69       162

    accuracy                           0.65      1438
   macro avg       0.65      0.64      0.64      1438
weighted avg       0.66      0.65      0.65      1438

[16:48:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [13:12<01:03, 63.30s/it]

              precision    recall  f1-score   support

         CpG       0.52      0.61      0.56       196
         FLA       0.51      0.48      0.49       143
         FSL       0.59      0.63      0.61       189
         LPS       0.58      0.67      0.62       149
         P3K       0.63      0.48      0.54       185
         PIC       0.78      0.72      0.74       165
         R84       0.79      0.84      0.81       249
         TNF       0.77      0.67      0.72       162

    accuracy                           0.65      1438
   macro avg       0.65      0.64      0.64      1438
weighted avg       0.65      0.65      0.65      1438

[16:49:07] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [13:31<00:00, 81.14s/it]

              precision    recall  f1-score   support

         CpG       0.48      0.55      0.52       196
         FLA       0.48      0.44      0.46       143
         FSL       0.57      0.61      0.59       189
         LPS       0.55      0.68      0.61       149
         P3K       0.63      0.50      0.56       185
         PIC       0.80      0.69      0.74       165
         R84       0.78      0.82      0.80       249
         TNF       0.73      0.67      0.70       162

    accuracy                           0.63      1438
   macro avg       0.63      0.62      0.62      1438
weighted avg       0.64      0.63      0.63      1438



In [14]:
d

0.4